# Implied Volatility

mplied volatility (IV) is one of the most important parameter in options pricing. IV is determined by the current market price of option contracts on a particular underlying asset. IV is commonly represented as a percentage that indicates the annualized expected one standard deviation range for the underlying asset implied from the option prices.

IV sigam is the volatility value that makes the Black Scholes value of the option equal to the traded price of the option. In the Black-Scholes model, volatility is the only parameter that can't be directly observed. All other parameters can be determined through market data and this parameter is determined by a numerical optimization technique given the Black-Scholes model.

In [2]:
# Data Manipulation
import pandas as pd
from numpy import *
from datetime import timedelta
import yfinance as yf
from tabulate import tabulate

# Math & Optimization
from scipy.stats import norm
from scipy.optimize import fsolve

# Plotting
import matplotlib.pyplot as plt
import cufflinks as cf
cf.set_config_file(offline=True)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
class BS:
    
    """
    This is a class for Options contract for pricing European options on stocks/index without dividends.
    
    Attributes: 
        spot          : int or float
        strike        : int or float 
        rate          : float
        dte           : int or float [days to expiration in number of years]
        volatility    : float
        callprice     : int or float [default None]
        putprice      : int or float [default None]
    """    
    
    def __init__(self, spot, strike, rate, dte, volatility, callprice=None, putprice=None):
        
        # Spot Price
        self.spot = spot
        
        # Option Strike
        self.strike = strike
        
        # Interest Rate
        self.rate = rate
        
        # Days To Expiration
        self.dte = dte
        
        # Volatlity
        self.volatility = volatility
        
        # Callprice # mkt price
        self.callprice = callprice
        
        # Putprice # mkt price
        self.putprice = putprice
            
        # Utility 
        self._a_ = self.volatility * self.dte**0.5
        
        if self.strike == 0:
            raise ZeroDivisionError('The strike price cannot be zero')
        else:
            self._d1_ = (log(self.spot / self.strike) + \
                     (self.rate + (self.volatility**2) / 2) * self.dte) / self._a_
        
        self._d2_ = self._d1_ - self._a_
        
        self._b_ = e**-(self.rate * self.dte)
        
        
        # The __dict__ attribute
        '''
        Contains all the attributes defined for the object itself. It maps the attribute name to its value.
        '''
        for i in ['callPrice', 'putPrice', 'callDelta', 'putDelta', 'callTheta', 'putTheta', \
                  'callRho', 'putRho', 'vega', 'gamma', 'impvol']:
            self.__dict__[i] = None
        
        [self.callPrice, self.putPrice] = self._price()
        [self.callDelta, self.putDelta] = self._delta()
        [self.callTheta, self.putTheta] = self._theta()
        [self.callRho, self.putRho] = self._rho()
        self.vega = self._vega()
        self.gamma = self._gamma()
        self.impvol = self._impvol()
    
    # Option Price
    def _price(self):
        '''Returns the option price: [Call price, Put price]'''

        if self.volatility == 0 or self.dte == 0:
            call = maximum(0.0, self.spot - self.strike)
            put = maximum(0.0, self.strike - self.spot)
        else:
            call = self.spot * norm.cdf(self._d1_) - self.strike * e**(-self.rate * \
                                                                       self.dte) * norm.cdf(self._d2_)

            put = self.strike * e**(-self.rate * self.dte) * norm.cdf(-self._d2_) - \
                                                                        self.spot * norm.cdf(-self._d1_)
        return [call, put]

    # Option Delta
    def _delta(self):
        '''Returns the option delta: [Call delta, Put delta]'''

        if self.volatility == 0 or self.dte == 0:
            call = 1.0 if self.spot > self.strike else 0.0
            put = -1.0 if self.spot < self.strike else 0.0
        else:
            call = norm.cdf(self._d1_)
            put = -norm.cdf(-self._d1_)
        return [call, put]

    # Option Gamma
    def _gamma(self):
        '''Returns the option gamma'''
        return norm.pdf(self._d1_) / (self.spot * self._a_)

    # Option Vega
    def _vega(self):
        '''Returns the option vega'''
        if self.volatility == 0 or self.dte == 0:
            return 0.0
        else:
            return self.spot * norm.pdf(self._d1_) * self.dte**0.5 / 100

    # Option Theta
    def _theta(self):
        '''Returns the option theta: [Call theta, Put theta]'''
        call = -self.spot * norm.pdf(self._d1_) * self.volatility / (2 * self.dte**0.5) - self.rate * self.strike * self._b_ * norm.cdf(self._d2_)

        put = -self.spot * norm.pdf(self._d1_) * self.volatility / (2 * self.dte**0.5) + self.rate * self.strike * self._b_ * norm.cdf(-self._d2_)
        return [call / 365, put / 365]

    # Option Rho
    def _rho(self):
        '''Returns the option rho: [Call rho, Put rho]'''
        call = self.strike * self.dte * self._b_ * norm.cdf(self._d2_) / 100
        put = -self.strike * self.dte * self._b_ * norm.cdf(-self._d2_) / 100

        return [call, put]
    
    # Option Implied Volatility
    def _impvol(self):
        '''Returns the option implied volatility'''
        if (self.callprice or self.putprice) is None:
            return self.volatility
        else:
            def f(sigma):
                option = BS(self.spot,self.strike,self.rate,self.dte,sigma)
                if self.callprice:
                    return option.callPrice - self.callprice # f(x) = BS_Call - MarketPrice
                if self.putprice and not self.callprice:
                    return option.putPrice - self.putprice

            return maximum(1e-5, fsolve(f, 0.2)[0])

In [5]:
# Initialize option
option = BS(100,100,0.02,1,0.2,8)

header = ['Option Price', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho', 'IV']
table = [[option.callPrice, option.callDelta, option.gamma, option.callTheta, option.vega, option.callRho, option.impvol]]

print(tabulate(table,header))

  Option Price    Delta      Gamma      Theta      Vega       Rho        IV
--------------  -------  ---------  ---------  --------  --------  --------
       8.91604  0.57926  0.0195521  -0.013399  0.391043  0.490099  0.176572


In [6]:
option.impvol

0.17657213831399188

# Newton Method

In [7]:
def newton_iv(className, spot, strike, rate, dte, volatility, callprice=None, putprice=None):
    
    x0 = 1 # initial guess
    h = 0.001
    tolerance = 1e-7
    epsilon = 1e-14 # some kind of error or floor
    
    maxiter = 200
    
    if callprice:
        # f(x) = Black Scholes Call price - Market Price
        f = lambda x: eval(className)(spot, strike, rate, dte, x).callPrice - callprice
    if putprice:
        f = lambda x: eval(className)(spot, strike, rate, dte, x).putPrice - putprice
        
    for i in range(maxiter):
        y = f(x0)
        yprime = (f(x0+h) - f(x0-h))/(2*h) # central difference
        
        if abs(yprime)<epsilon:
            break # this is critial, because volatility cannot be negative
        x1 = x0 - y/yprime
        
        if (abs(x1-x0) <= tolerance*abs(x1)):
            break
        x0=x1
        
    return x1
        

In [8]:
opt = BS(100,100,0.02,1,0.2)

In [9]:
opt.callPrice

8.916037278572539

In [10]:
newton_iv('BS',100,100,0.02,1,0.2,callprice=8.916037278572539)

0.20000000000000015

In [11]:
newton_iv('BS',100,100,0.02,1,0.2,putprice=10)

0.27842040930050715

# Bisection Method

In [12]:
# Bisection Method
def bisection_iv(className, spot, strike, rate, dte, volatility, callprice=None, putprice=None, high=500.0, low=0.0):
    
    if callprice:
        price = callprice
    if putprice and not callprice:
        price = putprice
        
    tolerance = 1e-7
        
    for i in range(1000):
        mid = (high + low) / 2 # c= (a+b)/2
        if mid < tolerance:
            mid = tolerance
            
        if callprice:
            estimate = eval(className)(spot, strike, rate, dte, mid).callPrice # Blackscholes price
        if putprice:
            estimate = eval(className)(spot, strike, rate, dte, mid).putPrice
        
        if round(estimate,6) == price:
            break
        elif estimate > price: 
            high = mid # b = c
        elif estimate < price: 
            low = mid # a = c
    
    return mid

In [13]:
bisection_iv('BS',100,100,0.02,1,0.2,callprice=8)

0.17657213902566582

# SPY Option

Let's now retrieve SPY option price from Yahoo Finance using yfinance library and manipulated the dataframe using the above Black Scholes option pricing model that we created.

https://finance.yahoo.com/quote/SPY/options?date=1609372800&p=SPY

In [14]:
# Create a ticker object
spy = yf.Ticker('SPY')

In [15]:
# Get Expiry dates 
expiry_dates = sorted(spy.options)
expiry_dates[:5]

['2023-02-10', '2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16']

In [16]:
# retrieve from yahoo finance
spy_call = spy.option_chain().calls
spy_puts = spy.option_chain().puts

# save locally
# spy_call.to_csv('/Users/jason/Desktop/spy_call.csv')
# spy_puts.to_csv('/Users/jason/Desktop/spy_puts.csv')

In [20]:
# Filter call option for Mar 2021 Expiry
mar = spy.option_chain('2023-03-10')

# Filter calls for strike above 390
mar = mar.calls[mar.calls['strike']>390]
mar.set_index('strike', inplace=True)

# Check the filtered output
mar.iloc[:,:11].head()

,contractSymbol,lastTradeDate,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney
strike,,,,,,,,,,,
391.0,SPY230310C00391000,2023-02-09 14:30:01+00:00,26.92,0.0,0.0,0.0,0.0,1.0,144,0.00001,True
392.0,SPY230310C00392000,2023-02-09 19:27:19+00:00,21.15,0.0,0.0,0.0,0.0,5.0,161,0.00001,True
392.5,SPY230310C00392500,2023-02-09 14:30:01+00:00,25.61,0.0,0.0,0.0,0.0,1.0,235,0.00001,True
393.0,SPY230310C00393000,2023-02-08 15:15:09+00:00,24.49,0.0,0.0,0.0,0.0,10.0,244,0.00001,True
394.0,SPY230310C00394000,2023-02-07 17:55:30+00:00,24.38,0.0,0.0,0.0,0.0,1.0,81,0.00001,True


In [21]:
mar[['ask', 'impliedVolatility']][:425].iplot(secondary_y='impliedVolatility', mode='lines+markers', size=6, 
                                              xTitle='Strike Price', title='Option Call Price Vs Implied Volatility')

# IV Skew

In [22]:
# Spot prices
spy_spot = 392
rate = 0.015

# Set valuation and expiry dates
valuation_date = pd.Timestamp('2021-02-26')
expiration_date = pd.Timestamp('2021-03-31')
dte = (expiration_date-valuation_date)/timedelta(365) # time to maturity in years

# Filter dataframe with ltp
df = mar[['bid', 'ask']]  
df['mid'] = 0.5 * (df['bid']+df['ask']) # (ask+bid)/2

# Add strike column
df['strike'] = df.index

# Initialize the IV column
df['IV'] = 0.

for i in range(len(df)):
    df['IV'].iloc[i] = 100*BS(spy_spot,
                          df.strike.iloc[i],
                          rate, dte, 0.2,
                          callprice = df.mid.iloc[i]).impvol

df.head()

,bid,ask,mid,strike,IV
strike,,,,,
391.0,0.0,0.0,0.0,391.0,20.0
392.0,0.0,0.0,0.0,392.0,20.0
392.5,0.0,0.0,0.0,392.5,20.0
393.0,0.0,0.0,0.0,393.0,20.0
394.0,0.0,0.0,0.0,394.0,20.0


In [23]:
df['IV'][:425].iplot(mode='lines+markers', size=6, xTitle='Strike Price', title=' MAR 2021 Call IV')

# User Defined Function

In [24]:
def ivskew(spot, rate, value_date, expiry, dataframe):
    
    valuation_date = pd.Timestamp(value_date)
    expiration_date = pd.Timestamp(expiry)
    dte = (expiration_date-valuation_date)/timedelta(365)

    df = dataframe[['bid', 'ask']]
    df['mid'] = 0.5 * (df['bid']+df['ask'])
    df['strike'] = df.index
    df['IV'] = 0.

    for i in range(len(df)):
        df['IV'].iloc[i] = 100*BS(spot, df.strike.iloc[i], rate, dte, 0.2, callprice = df.mid.iloc[i]).impvol

    return df['IV']

In [26]:
# Set expiry list
expiry_list = ['2023-02-10', '2023-02-17']

# Initialise empty dataframe
call_close = pd.DataFrame()
call_vols = pd.DataFrame()

for expiry in expiry_list:
    
    sub_set = spy.option_chain(expiry).calls
    sub_set = sub_set[(sub_set.strike>390) & (sub_set.strike<405)]
    sub_set.set_index('strike', inplace=True)
    
    call_close[expiry] = sub_set['ask']
    call_vols[expiry] = ivskew(spy_spot, rate, '2021-03-26', expiry, sub_set)

In [27]:
# Verify Call IV
# call_vols.index = around(call_vols.index/335,3)
call_vols.iloc[:10]

,2023-02-10,2023-02-17
strike,,
391.0,20.0,20.0
392.0,20.0,20.0
393.0,20.0,20.0
394.0,20.0,20.0
395.0,20.0,20.0
396.0,20.0,20.0
397.0,20.0,20.0
398.0,20.0,20.0
399.0,20.0,20.0


# Visualize

In [28]:
call_close.iplot(mode='lines+markers', symbol=['circle-dot', 'square'], size=6, 
                 title='Option Call Price', xTitle='strike price')

In [29]:
call_vols[:400].iplot(mode='lines+markers', symbol=['circle-dot', 'square'], size=6, 
                 title='Expiry-wise IV Skew', xTitle='strike price')